# 🔐 XOR + LSB Steganografi – Mesaj Gizleme (String Anahtar Destekli)

In [1]:
from PIL import Image

def xor_sifrele(mesaj, anahtar):
    mesaj_bytes = mesaj.encode('utf-8')
    anahtar_bytes = anahtar.encode('utf-8')
    return bytes([b ^ anahtar_bytes[i % len(anahtar_bytes)] for i, b in enumerate(mesaj_bytes)])

def xorlu_mesajı_gizle(gorsel_yolu, mesaj, anahtar, kayit_adi="B_encoded_medium.png"):
    sifreli = xor_sifrele(mesaj, anahtar)
    binary_mesaj = ''.join(format(b, '08b') for b in sifreli) + '1111111111111111'

    img = Image.open(gorsel_yolu)
    if img.mode != 'RGB':
        img = img.convert('RGB')

    encoded = img.copy()
    data_idx = 0

    for y in range(encoded.size[1]):
        for x in range(encoded.size[0]):
            r, g, b = encoded.getpixel((x, y))
            rgb = [r, g, b]
            for i in range(3):
                if data_idx < len(binary_mesaj):
                    rgb[i] = (rgb[i] & ~1) | int(binary_mesaj[data_idx])
                    data_idx += 1
            encoded.putpixel((x, y), tuple(rgb))
            if data_idx >= len(binary_mesaj):
                encoded.save(kayit_adi)
                print(f"[✓] Mesaj başarıyla gömüldü: {kayit_adi}")
                return

In [ ]:
# Örnek kullanım
mesaj = "Bybit saldırısı başlatıldı."
anahtar = "gizlimetin"
xorlu_mesajı_gizle("B.png", mesaj, anahtar)

# 🔓 XOR + LSB Steganografi – Mesaj Çözme (Anahtar Doğrulama ile)

In [3]:
from PIL import Image

def xor_coz(sifreli_bytes, anahtar):
    anahtar_bytes = anahtar.encode('utf-8')
    try:
        return bytes([b ^ anahtar_bytes[i % len(anahtar_bytes)] for i, b in enumerate(sifreli_bytes)]).decode('utf-8')
    except UnicodeDecodeError:
        return "[❌] Yanlış anahtar girildi veya mesaj bozulmuş."

def xorlu_mesajı_çöz(gorsel_yolu, anahtar):
    img = Image.open(gorsel_yolu)
    if img.mode != 'RGB':
        img = img.convert('RGB')

    binary_data = ""
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            for color in img.getpixel((x, y)):
                binary_data += str(color & 1)

    byte_data = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if byte == '11111111' and binary_data[i+8:i+16] == '11111111':
            break
        byte_data.append(int(byte, 2))

    sifreli_bytes = bytes(byte_data)
    return xor_coz(sifreli_bytes, anahtar)

In [ ]:
# Örnek çözüm
anahtar = "gizlimetin"
print(xorlu_mesajı_çöz("B_encoded_medium.png", anahtar))